In [1]:
import process as pr
import model as md
import pandas as pd
import forest as fr
%load_ext autoreload
%autoreload 2

In [ ]:
# the raw training data
raw = pr.get_raw('train')
# the processed training data
train_p = pr.get_post('train')
# the random forest
rf = fr.get_forest()
# the processed test data
test_p = pr.get_post('test')
# predicting the probabilities and aggregating the data
final = md.prep_for_submission()